# Google Colab

You can use the button below to open this notebook in Google Colab. Note that changes made to the notebook in Colab will not be reflected in Github, nor can the notebook be saved on Colab without first making a copy. 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nikitalokhmachev-ai/radio-map-estimation-public/blob/main/notebooks/Evaluate_Model.ipynb)

If opened in Colab, set `using_colab` to `True` in the code block below, then run the second and (optionally) third blocks. The second block will install the needed version of joblib for the data scaler to be loaded, then clone the github repository into Colab's local storage in order to load the models and other functions. The third block will connect to Google Drive (user login required), which allows the Colab notebook to read and write data to the drive (e.g. training data or evaluation results).

In [ ]:
using_colab = False

In [ ]:
if using_colab:
    %cd /content/
    !rm -rf /content/radio-map-estimation-public
    !git clone https://github.com/nikitalokhmachev-ai/radio-map-estimation-public.git
    !pip install -q -r /content/radio-map-estimation-public/requirements.txt

In [ ]:
if using_colab:
    from google.colab import drive
    drive.mount('/content/drive')

# Check GPU

It is recommended to run this notebook with GPU support. If you have an Nvidea graphics card and drivers installed, the following block of code should show the details of the installed GPU.

In [ ]:
!nvidia-smi

# Untar Testing Data

In the code block below, specify the path to the saved testing data in tar format. This will untar the data into a folder of the same name in the parent directory of this notebook.

In [ ]:
!tar -xkf '/path/to/saved/tar/file' -C '/path/to/save/untarred/files'

# Import Packages

In [ ]:
import torch
import numpy as np

import os
import glob
import random
import pickle

In [ ]:
# Import model architectures and data structures

os.chdir('path/to/repository')
from test_utils import get_model_error

# Set Hyperparameters

In [ ]:
# Set random seed, define device

seed = 3
torch.manual_seed(seed)
torch.use_deterministic_algorithms(True)
np.random.seed(seed)
random.seed(seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Set batch size
test_batch_size = 1024

# Manually set values for buildings, unsampled locations, and sampled locations in the environment mask. 
# For the models in the PIMRC paper, these are set to "None", meaning they keep the default values of -1, 0, and 1 respectively.
building_value = None
unsampled_value = None
sampled_value = None

Set the folder to load trained models from. All models in the selected model_folder will be tested. Additionally create and specify a folder to save the results to, and specify the paths to the saved data and data scaler (located within this repository).

In [ ]:
# Specify folder containing trained models
model_folder = '/folder/with/trained/models'

# Specify path to untarred test data
test_data_folder = '/path/to/untarred/testing/data'

# Specify path to data scaler
scaler_path = 'scalers/minmax_scaler_zero_min134.joblib'

# Set folder to save current results
results_folder = '/folder/to/save/results'
if not os.path.exists(results_folder):
    os.makedirs(results_folder)

In [ ]:
model_names = glob.glob(os.path.join(model_folder, '*.pth'))
for model_path in model_names:
  error = get_model_error(test_data_folder, test_batch_size, model_path, scaler_path, building_value=building_value, sampled_value=sampled_value)
  filename = os.path.basename(model_path).split('.')[0] + '.pickle'
  with open(os.path.join(results_folder, filename), 'wb') as f:
    pickle.dump(error, f)